In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
'''This object entails the dynamics od a vehicle in the simulation'''
class Vehicle:
    def __init__(self, position, speed, max_speed, vehicle_length, vehicle_width, braking_prob):
        '''attributes of the vehicle'''
        self.position = position
        self.speed = speed
        self.max_speed = max_speed
        self.vehicle_length = vehicle_length
        self.vehicle_width = vehicle_width
        self.braking_prob = braking_prob

    def accelerate(self):
        if self.speed < self.max_speed:
            self.speed += 1

    def decelerate(self, headway_distance):
        if self.speed > headway_distance:
            self.speed = headway_distance

    def random_brake(self):
        if np.random.rand() < self.braking_prob and self.speed > 0:
            self.speed -= 1

In [7]:
class Road:
    def __init__(self, length, width): #attributes of the road
        self.length = length
        self.width = width
        self.road_state = np.zeros((self.length, self.width), dtype = int)

    def update_road_state(self, new_state):
        self.road_state = new_state

    def get_road_state(self):
        return self.road_state

In [13]:
# Create a Road object with a length of 20 and width of 2
road = Road(length=20, width=2)

# Get the current state of the road
road_array = road.get_road_state()

# Print the road array
print(road_array)

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


In [8]:
class Controller:
    def __init__(self, road, produce_prob, max_speed, braking_prob):
        self.road = road
        self.produce_prob = produce_prob
        self.max_speed = max_speed
        self.braking_prob = braking_prob
        self.vehicles = [] #empty list to store vehicles on the road

    def produce_vehicles(self):
        if np.random.rand() < self.produce_prob: #Determines the time to produce vehicles
            vehicle_length = np.random.choice([1,2]) #Randomly selects vehicle length (1 for Motorcycle, 2 for Car)
            if vehicle_length == 1:
                vehicle_width = 1 #Motorcycles have a dimension of 1x1
            else:
                vehicle_width = 2 #Motorcycles have a dimension of 2x2
            if np.sum(self.road.road_state[0:vehicle_length, 0:vehicle_width]) == 0: #Checks if the beginning of the road is available for the vehicle
                '''New vehicle is produced should the beginning portion of the road that fits the vehicle is empty'''
                new_vehicle = Vehicle(0, np.random.randint(1, self.max_speed + 1), self.max_speed, vehicle_length, vehicle_width, self.braking_prob==0.1)
                self.vehicles.append((0, 0, new_vehicle)) #puts the new vehicle on the list of vehicles
                self.road.road_state[0:vehicle_length, 0:vehicle_width] = 1 #updates the road state
                return
            else:
                pass

    def move_vehicles(self):
        new_road_state = np.zeros((self.road.width, self.road.length), dtype = int)
        new_vehicles = []

        for i, j, vehicle in self.vehicles:
            #Check if there's available space ahead of the vehicle to move
            if np.sum(self.road.road_state[i + vehicle.vehicle_length + vehicle.speed:i + vehicle.vehicle_length, j:j + vehicle.vehicle_width]) == 0:
                vehicle.accelerate()

                #Update the road state
                new_road_state[i + vehicle.vehicle_length+vehicle.speed:i+vehicle.vehicle_length, j:j + vehicle.vehicle_width] = 1

                #Append the updated vehicle position to the new_vehicles list
                new_vehicles.append((i + vehicle.speed, j, vehicle)) #you move the position of the vehicle of its speed cells
                #You stay in your line, that's why j is not changed on the vehicle's position

            else:
                #If there's no space ahead, decelerate the vehicle based on the headway distance
                headway_distance = np.sum(self.road.road_state[i + vehicle.vehicle_length  + vehicle.speed:i + vehicle.vehicle_length, j:j+vehicle.vehicle_width])
                vehicle.decelerate(headway_distance)

                #Update the road state to mark the area occupied by the vehicle
                new_road_state[i + vehicle.vehicle_length+ vehicle.speed:i + vehicle.vehicle_length, j:j+vehicle.vehicle_width] = 1

                #Append the updates vehicle position to the new vehicles list
                new_vehicles.append((i+vehicle.speed, j, vehicle))

            self.road.update_road_state(new_road_state)
            #Update the road state with the new state

            #Update the lost of vehicles with the new positions
            self.vehicles = [(i, j, vehicle) for i, j, vehicle in new_vehicles if i+vehicle.vehicle_length < self.road.length]


    def remove_vehicles(self):
        self.road.road_state = np.roll(self.road.road_state, -1, axis=0)
        self.road.road_state[-1, :] = 0
        

In [9]:
def visualize_simulation(road, controller, T):
    road_states = []

    for t in range(T):
        controller.produce_vehicles()
        controller.move_vehicles()
        controller.remove_vehicles()
        road_states.append(road.get_road_state().copy())

    for t, road_state in enumerate(road_states):
        print(f"Road State at Time Step {t}:")
        print(road_state)
        print()

# Example usage:
# Parameters
length = 20  # Length of the road
width = 2   # Width of the road
production_prob = 0.3  # Probability of producing vehicles
max_speed = 5  # Maximum speed of vehicles
braking_prob = 0.05  # Probability of random braking
T = 20  # Number of timesteps

# Initialization
road = Road(length, width)
controller = Controller(road, production_prob, max_speed, braking_prob)

visualize_simulation(road, controller, T)

Road State at Time Step 0:
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]

Road State at Time Step 1:
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]

Road State at Time Step 2:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Road State at Time Step 3:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Road State at Time Step 4:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Road State at Time Step 5:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Road State at Time Step 6:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

Road State at Time Step 7:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0